# Talk to Alpaca-LoRA

This notebook contains minimal code for running [Alpaca-LoRA](https://github.com/tloen/alpaca-lora/) for demonstration purposes. Please check the repo for more details.

In [1]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q gradio

     ------------------------------------- 104.3/104.3 MB 19.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\programdata\\anaconda3\\lib\\site-packages\\tqdm-4.50.2.dist-info\\direct_url.json'
Consider using the `--user` option or check the permissions.



  Cloning https://github.com/huggingface/transformers.git to c:\users\cpprh\appdata\local\temp\pip-req-build-32a5tpie
  Resolved https://github.com/huggingface/transformers.git to commit b0a78091a5b2f7e872140cf2d3795e4c56c9c95d
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
     ---------------------------------------- 3.5/3.5 MB 44.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.29.0.dev0-py3-none-any.whl size=7010083 sha256=1df42f6748e3c99ab7e58b4419b076e4e469d7f5e384105aa369fb5eb80cf815
  Stored in directory: C:\Users\cpprh\AppData\Local\Temp\pip-ephem-wheel-cache-im49zcxh\wheels\05\0a\97\64ae47c27ba95fae2cb5838e7

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\cpprh\AppData\Local\Temp\pip-req-build-32a5tpie'
ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\programdata\\anaconda3\\lib\\site-packages\\tokenizers-0.10.3.dist-info\\INSTALLER'
Consider using the `--user` option or check the permissions.

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\programdata\\anaconda3\\lib\\site-packages\\caffe2\\contrib\\aten\\aten_test.py'
Consider using the `--user` option or check the permissions.



ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\programdata\\anaconda3\\lib\\site-packages\\idna-2.6.dist-info\\DESCRIPTION.rst'
Consider using the `--user` option or check the permissions.



In [ ]:
!git clone https://github.com/tloen/alpaca-lora

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 268 (delta 6), reused 1 (delta 0), pack-reused 256
Receiving objects: 100% (268/268), 13.77 MiB | 7.17 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [ ]:
cd alpaca-lora

/content/alpaca-lora


In [2]:
!python generate.py

Traceback (most recent call last):
  File "generate.py", line 2, in <module>
    from peft import PeftModel
ModuleNotFoundError: No module named 'peft'


In [5]:
import torch
from peft import PeftModel
import transformers
import gradio as gr
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

# tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-13b-hf")
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

# BASE_MODEL = "decapoda-research/llama-13b-hf"
BASE_MODEL = "decapoda-research/llama-7b-hf"
# LORA_WEIGHTS = "samwit/alpaca13B-lora"
LORA_WEIGHTS = "tloen/alpaca-lora-7b"


model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)


ModuleNotFoundError: No module named 'torch.distributed.algorithms'

In [ ]:
LORA_WEIGHTS = "samwit/alpaca7B-lora"
model = PeftModel.from_pretrained(model, LORA_WEIGHTS, torch_dtype=torch.float16)

ValueError: ignored

In [ ]:

def generate_prompt(instruction, input=None):
    if input:
        return f"""아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절하게 완료하는 응답을 작성하세요.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""아래는 작업을 설명하는 지침입니다. 요청을 적절하게 완료하는 응답을 작성하세요.

### Instruction:
{instruction}

### Response:"""


model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)


def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()


gr.Interface(
    fn=evaluate,
    inputs=[
        gr.components.Textbox(
            lines=2, label="Instruction", placeholder="Tell me about alpacas."
        ),
        gr.components.Textbox(lines=2, label="Input", placeholder="none"),
        gr.components.Slider(minimum=0, maximum=1, value=0.1, label="Temperature"),
        gr.components.Slider(minimum=0, maximum=1, value=0.75, label="Top p"),
        gr.components.Slider(minimum=0, maximum=100, step=1, value=40, label="Top k"),
        gr.components.Slider(minimum=1, maximum=4, step=1, value=4, label="Beams"),
        gr.components.Slider(
            minimum=1, maximum=2000, step=1, value=128, label="Max tokens"
        ),
    ],
    outputs=[
        gr.inputs.Textbox(
            lines=5,
            label="Output",
        )
    ],
    title="🦙🌲 Alpaca-LoRA",
    description="Alpaca-LoRA is a 7B-parameter LLaMA model finetuned to follow instructions. It is trained on the [Stanford Alpaca](https://github.com/tatsu-lab/stanford_alpaca) dataset and makes use of the Huggingface LLaMA implementation. For more information, please visit [the project's website](https://github.com/tloen/alpaca-lora).",
).launch(share=True)

